In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from rich.pretty import pprint
from rich.markdown import Markdown

load_dotenv()


MODEL = os.getenv("MODEL", "gemini-2.5-flash-lite")
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
llm = ChatOpenAI(
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,  # type: ignore
    temperature=0,
)

print(llm.invoke("Hi!").content)

Hi there! How can I help you today?


In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage

from langgraph.types import Send, Command

In [4]:
from typing import TypedDict, Annotated, NotRequired, Literal
import operator

class AgentState(TypedDict):
    topic: str
    task: Literal["joke", "poem", "story"]
    prompt: NotRequired[str]
    response: NotRequired[str]

In [5]:
def _init_llm(state: AgentState)->Command[Literal["generate_response"]]:
    topic = state["task"]
    if topic == "joke":
        prompt = f"Tell me a joke about {topic}"
    elif topic == "poem":
        prompt = f"Write a poem about {topic}"
    elif topic == "story":
        prompt = f"Write a short story about {topic}"

    return Command(
        update={"prompt": prompt},
        goto="generate_response",
    )

def _gen_response(state: AgentState):
    response = llm.invoke([HumanMessage(
        content=state.get('prompt', '')
    )])

    return {"response": response.content}

In [6]:
graph = StateGraph(AgentState)

graph.add_node("generate_response", _gen_response)
graph.add_node("init_llm", _init_llm)

graph.add_edge(START, "init_llm")

graph.add_edge("generate_response", END)

agent = graph.compile()
print(agent.get_graph().draw_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
          *            
          *            
          *            
    +----------+       
    | init_llm |       
    +----------+       
          .            
          .            
          .            
+-------------------+  
| generate_response |  
+-------------------+  
          *            
          *            
          *            
     +---------+       
     | __end__ |       
     +---------+       


In [7]:
response = agent.invoke({"topic": "monsoon", "task": "joke"})

In [8]:
pprint(response)

{
│   'topic': 'monsoon',
│   'task': 'joke',
│   'prompt': 'Tell me a joke about joke',
│   'response': 'Why did the joke break up with the punchline?\n\nBecause it felt like the punchline was always *telling* it what to do!'
}

In [14]:
class AgentState(TypedDict):
    messages: NotRequired[Annotated[list, operator.add]]

In [15]:
def _init_llm(state: AgentState):
    system_prompt = (
        "Your are an helpful chatbot with strong memory and recall capabilities."
    )
    messages = [SystemMessage(content=system_prompt)]

    return {"messages": messages}


def _call_model(state: AgentState):
    messages = state.get("messages", [])
    response = llm.invoke(messages)
    print("Assistant response: ", end="")
    print(response.content)
    messages.append(response)

    return {"messages": messages}


def _should_continue(state: AgentState) -> Command[Literal["call_model", "__end__"]]:
    user_input = str(input("Enter your Query(use `quit` to exit): "))
    if user_input.lower() == "quit":
        return Command(goto="__end__")
    else:
        messages = [HumanMessage(content=user_input)]
        return Command(update={"messages": messages}, goto="call_model")

In [16]:
graph = StateGraph(AgentState)

graph.add_node("init_llm", _init_llm)
graph.add_node("call_model", _call_model)
graph.add_node("should_continue", _should_continue)

graph.add_edge(START, "init_llm")
graph.add_edge("init_llm", "should_continue")
graph.add_edge("call_model", "should_continue")

agent = graph.compile()
print(agent.get_graph().draw_ascii())

            +-----------+             
            | __start__ |             
            +-----------+             
                   *                  
                   *                  
                   *                  
             +----------+             
             | init_llm |             
             +----------+             
                   *                  
                   *                  
                   *                  
         +-----------------+          
         | should_continue |          
         +-----------------+          
            ***         ...           
           *               .          
         **                 ..        
+------------+           +---------+  
| call_model |           | __end__ |  
+------------+           +---------+  


In [19]:
response = agent.invoke({})

In [20]:
pprint(response["messages"])

[
│   SystemMessage(
│   │   content='Your are an helpful chatbot with strong memory and recall capabilities.',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   )
]